# Glassnode Data

Source: https://glassnode.com

### Import

In [13]:
import json
import requests
import pandas as pd
from datetime import date
from pathlib import Path 

### Get data

#### Get list of available data

In [2]:
# insert your API key here
API_KEY = '25cC9ajq5eVNBgT2hsQXRNEUEY4'
url = 'https://api.glassnode.com/v2/metrics/endpoints'

# make API request
res = requests.get(url,
    params={'a': 'BTC', 'api_key': API_KEY})

# Filter accessible data
df_endpoints = pd.read_json(res.text)
df_endpoints = df_endpoints[df_endpoints.tier == 1]

# Filter on Bitcoin data
df_endpoints['BTC'] = df_endpoints.apply(lambda row: row.assets[0]['symbol'] == 'BTC', axis=1)
df_endpoints = df_endpoints[df_endpoints.BTC == True]

# Get Cathegory/Name
df_endpoints['Cathegory'] = df_endpoints.apply(lambda row: row.path.split('/')[-2], axis=1)
df_endpoints['Name'] = df_endpoints.apply(lambda row: row.path.split('/')[-1], axis=1)

# Filter on cathegories
ignor_cathegories = ['institutions','market','supply','mining']
df_endpoints = df_endpoints[~df_endpoints.Cathegory.isin(ignor_cathegories)]

# Get url
df_endpoints['url'] = df_endpoints.apply(lambda row: 'https://api.glassnode.com'+row.path, axis=1)

#### Load available data

In [7]:
df_metadata = pd.DataFrame()

for i, endpoints in df_endpoints.iterrows():
    name = endpoints.Name
    cathegory = endpoints.Cathegory
    url = endpoints.url
    
    # make API request
    res_endpoint = requests.get(url, params={'a': 'BTC', 'api_key': API_KEY})
    
    try:
        # convert to pandas dataframe
        df_endpoint = pd.read_json(res_endpoint.text, convert_dates=['t'])
        df_endpoint = df_endpoint.rename(columns={'v': '/'.join([cathegory,name])})
        df_endpoint = df_endpoint.set_index(['t'])
        
        if df_endpoint.isna().sum().values[0] > 0:
            print(f'Error - NAN: {cathegory} | {name}')
            continue
        df_metadata = pd.concat([df_metadata, df_endpoint], join="outer", axis=1)
    except:
        print(f'Error - OTHER: {cathegory} | {name}')

Error - NAN: transactions | size_mean
Error - NAN: transactions | size_sum
Error - OTHER: indicators | difficulty_ribbon
Error - OTHER: blockchain | utxo_count
Error - OTHER: blockchain | block_count
Error - OTHER: blockchain | block_interval_mean
Error - OTHER: blockchain | block_interval_median
Error - OTHER: blockchain | block_size_mean
Error - OTHER: blockchain | block_size_sum


### Data engineering

In [8]:
today = date.today()
df_metadata = df_metadata[df_metadata.index < str(today)]
df_metadata = df_metadata.dropna(axis='columns')

### Some info on the dataset

In [9]:
display(df_metadata.head())
display(df_metadata.shape)

,addresses/count,addresses/sending_count,addresses/receiving_count,addresses/active_count,addresses/new_non_zero_count,transactions/count,transactions/rate,transactions/transfers_volume_sum,transactions/transfers_volume_mean,transactions/transfers_volume_median,...,fees/volume_mean,fees/volume_median,blockchain/utxo_created_count,blockchain/utxo_spent_count,blockchain/utxo_created_value_sum,blockchain/utxo_spent_value_sum,blockchain/utxo_created_value_mean,blockchain/utxo_spent_value_mean,blockchain/utxo_created_value_median,blockchain/utxo_spent_value_median
t,,,,,,,,,,,,,,,,,,,,,
2009-01-03,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,50.0,0.0,50.0,0.0,50.0,0.0
2009-01-04,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-05,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-06,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-07,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(4841, 21)

In [10]:
display(df_metadata.index.min())
display(df_metadata.index.max())

Timestamp('2009-01-03 00:00:00', freq='D')

Timestamp('2022-04-05 00:00:00', freq='D')

In [11]:
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4841 entries, 2009-01-03 to 2022-04-05
Freq: D
Data columns (total 21 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   addresses/count                       4841 non-null   float64
 1   addresses/sending_count               4841 non-null   float64
 2   addresses/receiving_count             4841 non-null   float64
 3   addresses/active_count                4841 non-null   float64
 4   addresses/new_non_zero_count          4841 non-null   float64
 5   transactions/count                    4841 non-null   float64
 6   transactions/rate                     4841 non-null   float64
 7   transactions/transfers_volume_sum     4841 non-null   float64
 8   transactions/transfers_volume_mean    4841 non-null   float64
 9   transactions/transfers_volume_median  4841 non-null   float64
 10  fees/volume_sum                       4841 non-null   floa

### Save data

In [12]:
filepath = Path('../csv/glassnode_onchain_data.csv')

NameError: name 'Path' is not defined

In [ ]:
df.to_csv(filepath)